In [1]:
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set() 
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer

from datetime import datetime, timedelta
from alive_progress import alive_bar, alive_it 
import random

# import crawler, team_rankings_crawler_and_scraper, matches_stats_scraper

matches_dataset = pd.read_csv("all_matches_dataset.csv")
matches_dataset

,match_url,match_date,winner,rw_diff,map,t1_name,t1_id,t1_rank,t1_total_rw,t1_fh_rw,...,t2p5_kills,t2p5_hskills,t2p5_assists,t2p5_fassists,t2p5_deaths,t2p5_kdratio,t2p5_kddiff,t2p5_adr,t2p5_fkdiff,t2p5_rating
0,/stats/matches/mapstatsid/46584/hellraisers-vs...,2017-05-17,t1,14,train,hellraisers,5310,8.0,16,13,...,5.0,2.0,1.0,0.0,17.0,0.333,-12.0,35.9,-3.0,0.26
1,/stats/matches/mapstatsid/62534/envy-vs-north,2018-03-08,t1,2,cobblestone,north,7533,13.0,16,8,...,15.0,9.0,1.0,0.0,22.0,0.633,-7.0,58.5,-3.0,0.78
2,/stats/matches/mapstatsid/61361/fnatic-vs-gambit,2018-02-17,t1,8,inferno,fnatic,4991,5.0,16,10,...,7.0,5.0,3.0,1.0,22.0,0.458,-15.0,49.2,-3.0,0.37
3,/stats/matches/mapstatsid/57159/liquid-vs-cloud9,2017-11-12,t2,-11,overpass,liquid,5973,8.0,5,3,...,16.0,5.0,3.0,0.0,16.0,0.714,0.0,67.6,0.0,1.00
4,/stats/matches/mapstatsid/123194/dbl-poney-vs-ago,2021-07-04,t1,2,ancient,dbl-poney,11003,25.0,16,6,...,15.0,10.0,2.0,0.0,22.0,0.567,-7.0,64.6,-1.0,0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41562,/stats/matches/mapstatsid/82789/uruguay-vs-sprout,2019-03-14,t2,-5,dust2,uruguay,9855,NaN,11,8,...,12.0,6.0,7.0,5.0,16.0,0.630,-4.0,43.9,-1.0,0.78
41563,/stats/matches/mapstatsid/127591/lese-vs-nebula,2021-09-28,t1,5,mirage,nebula,11353,NaN,16,13,...,14.0,6.0,4.0,1.0,20.0,0.667,-6.0,51.8,-4.0,0.70
41564,/stats/matches/mapstatsid/126396/rbg-vs-bad-ne...,2021-09-11,t1,9,nuke,bad-news-bears,9799,NaN,16,13,...,10.0,2.0,5.0,0.0,19.0,0.522,-9.0,58.3,-1.0,0.67
41565,/stats/matches/mapstatsid/87687/the-final-trib...,2019-06-10,t1,7,dust2,5balls,8589,NaN,16,9,...,10.0,3.0,2.0,1.0,18.0,0.600,-8.0,43.2,0.0,0.68


In [18]:
ranks_vs_winner_df = matches_dataset.loc[:, ["winner", "t1_rank", "t2_rank"]]
t1_won = matches_dataset.loc[(matches_dataset["winner"] == "t1"), ["t1_rank", "t2_rank"]].fillna(31)
t2_won = matches_dataset.loc[(matches_dataset["winner"] == "t2"), ["t1_rank", "t2_rank"]].fillna(31)
l = []
for index, row in t1_won.iterrows():
    l.append(row["t1_rank"] - row["t2_rank"])



# fig = plt.figure(figsize=(16, 9))

# plt.scatter(t1_won["t1_rank"], t1_won["t2_rank"])

# plt.show()

[-19.0,
 -15.0,
 -6.0,
 1.0,
 -3.0,
 -26.0,
 18.0,
 4.0,
 8.0,
 -1.0,
 -18.0,
 7.0,
 -4.0,
 14.0,
 19.0,
 4.0,
 -5.0,
 10.0,
 1.0,
 10.0,
 -11.0,
 -10.0,
 -7.0,
 -3.0,
 -21.0,
 -13.0,
 -6.0,
 -16.0,
 4.0,
 4.0,
 -14.0,
 -4.0,
 -13.0,
 16.0,
 -17.0,
 -25.0,
 8.0,
 -13.0,
 -6.0,
 -4.0,
 -19.0,
 3.0,
 2.0,
 -9.0,
 -5.0,
 -4.0,
 -25.0,
 -3.0,
 4.0,
 -2.0,
 2.0,
 -7.0,
 -13.0,
 -17.0,
 -20.0,
 -16.0,
 7.0,
 1.0,
 -7.0,
 -12.0,
 -25.0,
 16.0,
 -16.0,
 -1.0,
 -26.0,
 -7.0,
 -9.0,
 5.0,
 1.0,
 -11.0,
 12.0,
 12.0,
 -3.0,
 3.0,
 -5.0,
 -23.0,
 6.0,
 -11.0,
 7.0,
 19.0,
 2.0,
 -12.0,
 2.0,
 -1.0,
 7.0,
 -11.0,
 8.0,
 5.0,
 27.0,
 -1.0,
 -3.0,
 -22.0,
 -8.0,
 -26.0,
 15.0,
 -19.0,
 -3.0,
 -1.0,
 1.0,
 -4.0,
 -9.0,
 14.0,
 -11.0,
 -9.0,
 -12.0,
 16.0,
 -6.0,
 -3.0,
 -14.0,
 2.0,
 -15.0,
 -9.0,
 -19.0,
 0.0,
 9.0,
 -14.0,
 -1.0,
 -13.0,
 -2.0,
 -18.0,
 -1.0,
 0.0,
 -14.0,
 -25.0,
 7.0,
 -18.0,
 -10.0,
 -25.0,
 8.0,
 -21.0,
 4.0,
 -20.0,
 11.0,
 6.0,
 7.0,
 -19.0,
 -3.0,
 0.0,
 27.0,
 1.0,
 4.0,
 9.